In [1]:
import sys
sys.path.insert(0, '../')
import numpy as np
%reload_ext autoreload
%autoreload 1
%aimport allel
%aimport allel.model
%aimport allel.bcolz
allel.__version__

'0.15.0.dev3'

In [2]:
# setup an array of genotype calls
shape = n_variants, n_samples, ploidy = 50000, 1000, 2
data = np.zeros(shape, dtype='i1')
# simulate some mutations
n_alleles = n_variants * n_samples * ploidy
idx = np.random.randint(0, (n_alleles//2)-1, size=n_alleles//20)
data[:, :, 1].reshape((-1))[idx] = 1
data[:, :, 0].reshape((-1))[idx[:n_alleles//200]] = 1
g = allel.model.GenotypeArray(data, copy=False)

In [3]:
g

,0,1,2,3,4,...,995,996,997,998,999
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/1
1,0/1,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/1,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/1,0/0,0/0,0/1
4,0/0,0/0,0/0,0/0,0/1,...,0/0,0/0,0/0,0/0,0/0


In [4]:
g.take([0, 1, 2], axis=1)

,0,1,2
0,0/0,0/0,0/0
1,0/1,0/0,0/0
2,0/0,0/0,0/0
3,0/0,0/0,0/0
4,0/0,0/0,0/0


In [5]:
h = g.to_haplotypes()
h

,0,1,2,3,4,...,1995,1996,1997,1998,1999
0,0,0,0,0,0,...,0,0,0,0,1
1,0,1,0,0,0,...,0,0,0,0,0
2,0,0,0,0,0,...,0,0,0,0,0
3,0,0,0,0,0,...,0,0,0,0,1
4,0,0,0,0,0,...,0,0,0,0,0


In [6]:
h.take([0, 1, 2], axis=1)

,0,1,2
0,0,0,0
1,0,1,0
2,0,0,0
3,0,0,0
4,0,0,0


In [7]:
gc = allel.bcolz.GenotypeCArray(g)
gc

,0,1,2,3,4,...,995,996,997,998,999
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/1
1,0/1,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/1,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/1,0/0,0/0,0/1
4,0/0,0/0,0/0,0/0,0/1,...,0/0,0/0,0/0,0/0,0/0


In [8]:
hc = gc.to_haplotypes()
hc

,0,1,2,3,4,...,1995,1996,1997,1998,1999
0,0,0,0,0,0,...,0,0,0,0,1
1,0,1,0,0,0,...,0,0,0,0,0
2,0,0,0,0,0,...,0,0,0,0,0
3,0,0,0,0,0,...,0,0,0,0,1
4,0,0,0,0,0,...,0,0,0,0,0


In [9]:
ac = g.count_alleles()
ac

,0,1
0,1894,106
1,1888,112
2,1906,94
3,1907,93
4,1904,96


In [10]:
gac = gc.count_alleles()
gac

,0,1
0,1894,106
1,1888,112
2,1906,94
3,1907,93
4,1904,96
